In [1]:
from uuid import uuid4

In [2]:
from modelcluster.models import ClusterableModel, get_all_child_relations

In [3]:
post = Post.objects.get(pk=533)

In [4]:
body = post.body.raw_data

In [5]:
body[0]["value"][2]["value"]["gallery"]

[{'id': '4665bbce-8b97-42ae-ab2e-04ee6f9f38ea', 'type': 'item', 'value': 2501},
 {'id': '35a2664e-c799-4374-807e-d921a7c9c31b', 'type': 'item', 'value': 2500},
 {'id': '1782b042-fe98-4937-b0e0-1b3bc72b428c', 'type': 'item', 'value': 2499},
 {'id': '57dff242-25be-4575-9544-7063e714bf53', 'type': 'item', 'value': 2498},
 {'id': '7466e1c0-1a87-4b88-a31e-6be01f32297d', 'type': 'item', 'value': 2497}]

In [6]:
#post.save()

In [7]:
#ReferenceIndex.create_or_update_for_object(post)

In [8]:
references = list(ReferenceIndex._extract_references_from_object(post))

In [9]:
references

[(65,
  '193',
  'body.overview.video',
  'body.96178f8f-80b4-41ce-9c60-a490a9e94a50.f170f3ee-0b52-4baf-8806-1d70d9b2aa4d'),
 (57, '127', 'tagged_items.item.tag', 'tagged_items.587.tag')]

In [10]:
for to_content_type_id, to_object_id, model_path, content_path in references:
    print(to_content_type_id, to_object_id)

65 193
57 127


In [11]:
def extract_references_from_object_full(object):
    for field in object._meta.get_fields():
        if field.is_relation and field.many_to_one:
            if getattr(field, "wagtail_reference_index_ignore", False):
                continue

            if getattr(
                field.related_model, "wagtail_reference_index_ignore", False
            ):
                continue

            if isinstance(field, (ParentalKey, GenericRel)):
                continue

            if isinstance(field, GenericForeignKey):
                ct_field = object._meta.get_field(field.ct_field)
                fk_field = object._meta.get_field(field.fk_field)
                ct_value = ct_field.value_from_object(object)
                fk_value = fk_field.value_from_object(object)

                if ct_value is not None and fk_value is not None:
                    # The content type ID referenced by the GenericForeignKey might be a subclassed
                    # model, but the reference index requires us to index it under the base model's
                    # content type, as that's what will be used for lookups. So, we need to convert
                    # the content type back to a model class so that _get_base_content_type can
                    # select the appropriate superclass if necessary, before converting back to a
                    # content type.
                    model = ContentType.objects.get_for_id(ct_value).model_class()
                    yield (
                        ReferenceIndex._get_base_content_type(model).id,
                        str(fk_value),
                        field.name,
                        field.name,
                    )

                continue

            if isinstance(field, GenericRel):
                continue

            value = field.value_from_object(object)
            if value is not None:
                yield (
                    ReferenceIndex._get_base_content_type(field.related_model).id,
                    str(value),
                    field.name,
                    field.name,
                )

        if hasattr(field, "extract_references"):
            value = field.value_from_object(object)
            if value is not None:
                yield from (
                    (
                        ReferenceIndex._get_base_content_type(to_model).id,
                        to_object_id,
                        f"{field.name}.{model_path}",
                        f"{field.name}.{content_path}",
                    )
                    for to_model, to_object_id, model_path, content_path in field.extract_references(
                        value
                    )
                )

    # Extract references from child relations
    if isinstance(object, ClusterableModel):
        for child_relation in get_all_child_relations(object):
            relation_name = child_relation.get_accessor_name()
            child_objects = getattr(object, relation_name).all()

            for child_object in child_objects:
                yield from (
                    (
                        to_content_type_id,
                        to_object_id,
                        f"{relation_name}.item.{model_path}",
                        f"{relation_name}.{str(child_object.id)}.{content_path}",
                    )
                    for to_content_type_id, to_object_id, model_path, content_path in ReferenceIndex._extract_references_from_object(
                        child_object
                    )
                )


def extract_references_from_object(object):
    for field in object._meta.get_fields():
        if field.is_relation and field.many_to_one:
            if getattr(field, "wagtail_reference_index_ignore", False):
                continue

            if getattr(
                field.related_model, "wagtail_reference_index_ignore", False
            ):
                continue

            if isinstance(field, (ParentalKey, GenericRel)):
                continue

            if isinstance(field, GenericForeignKey):
                ct_field = object._meta.get_field(field.ct_field)
                fk_field = object._meta.get_field(field.fk_field)
                ct_value = ct_field.value_from_object(object)
                fk_value = fk_field.value_from_object(object)

                if ct_value is not None and fk_value is not None:
                    # The content type ID referenced by the GenericForeignKey might be a subclassed
                    # model, but the reference index requires us to index it under the base model's
                    # content type, as that's what will be used for lookups. So, we need to convert
                    # the content type back to a model class so that _get_base_content_type can
                    # select the appropriate superclass if necessary, before converting back to a
                    # content type.
                    model = ContentType.objects.get_for_id(ct_value).model_class()
                    yield (
                        ReferenceIndex._get_base_content_type(model).id,
                        str(fk_value),
                        field.name,
                        field.name,
                    )

                continue

            if isinstance(field, GenericRel):
                continue

            value = field.value_from_object(object)
            if value is not None:
                yield (
                    ReferenceIndex._get_base_content_type(field.related_model).id,
                    str(value),
                    field.name,
                    field.name,
                )

        if hasattr(field, "extract_references"):
            print("has extract references!")
            print("field: ", field)
            value = field.value_from_object(object)
            if value is not None:
                print("value: ", repr(value))
                yield from (
                    (
                        ReferenceIndex._get_base_content_type(to_model).id,
                        to_object_id,
                        f"{field.name}.{model_path}",
                        f"{field.name}.{content_path}",
                    )
                    for to_model, to_object_id, model_path, content_path in field.extract_references(
                        value
                    )
                )

In [12]:
for reference in extract_references_from_object(post):
    print(reference)

has extract references!
field:  cast.Post.body
value:  <StreamValue [<block overview: <StreamValue [<block paragraph: <wagtail.rich_text.RichText object at 0x12b3f9a60>>, <block video: 193>, <block gallery: {'layout': 'default', 'gallery': [{'id': '4665bbce-8b97-42ae-ab2e-04ee6f9f38ea', 'type': 'item', 'value': 2501}, {'id': '35a2664e-c799-4374-807e-d921a7c9c31b', 'type': 'item', 'value': 2500}, {'id': '1782b042-fe98-4937-b0e0-1b3bc72b428c', 'type': 'item', 'value': 2499}, {'id': '57dff242-25be-4575-9544-7063e714bf53', 'type': 'item', 'value': 2498}, {'id': '7466e1c0-1a87-4b88-a31e-6be01f32297d', 'type': 'item', 'value': 2497}]}>]>>]>
(65, '193', 'body.overview.video', 'body.96178f8f-80b4-41ce-9c60-a490a9e94a50.f170f3ee-0b52-4baf-8806-1d70d9b2aa4d')


In [14]:
post.body.extract_references(post.body)

AttributeError: 'StreamValue' object has no attribute 'extract_references'